### INFO284 Machine Learning Exam, spring 2024

#### Importing and versioncontrol for relevant libraries

In [116]:
import sys
print("Python version: {}".format(sys.version))
import pandas as pd
print("pandas version: {}".format(pd.__version__))
import matplotlib
import matplotlib.pyplot as plt
print("matplotlib version: {}".format(matplotlib.__version__))
import numpy as np
print("NumPy version: {}".format(np.__version__))
import scipy as sp
print("SciPy version: {}".format(sp.__version__))
import IPython
print("IPython version: {}".format(IPython.__version__))
import sklearn
print("scikit-learn version: {}".format(sklearn.__version__))
import seaborn as sns
print("seaborn version: {}".format(sns.__version__))

Python version: 3.12.2 (tags/v3.12.2:6abddd9, Feb  6 2024, 21:26:36) [MSC v.1937 64 bit (AMD64)]
pandas version: 2.2.1
matplotlib version: 3.8.3
NumPy version: 1.26.4
SciPy version: 1.12.0
IPython version: 8.21.0
scikit-learn version: 1.4.1.post1
seaborn version: 0.13.2


#### Importing the dataset

In [117]:
filePath = 'elektronisk-rapportering-ers-2018-fangstmelding-dca-simple.csv'
# Keep in mind that the file is encoded in UTF-8 so it will only work if you have the correct version of pandas.
df = pd.read_csv(filePath, encoding="UTF-8" , delimiter=";")
print(f"Before pre-processing the dataset has {df.shape[1]} columns and {df.shape[0]} rows")


Before pre-processing the dataset has 45 columns and 305434 rows


#### Step 1: chosing target and pre-prossesing
After taking some time to understand the data we have chosen our target features to be the catches of Hyse, Torsk and Sei. Next we must pre-prosess the data into suitable forms for modeling, this might take different forms as we try different ways to model it. The key skill here will therefor be the ability to manipulate the pandas dataframe.

Ways we might process the data:
1. Remove uncessary columns.
2. Collapse columns that say the same thing into one column.
3. Remove or manipulate non-sensical or missing values.
4. Transform values to work with our model.

*Question: should we also show how we went about understanding the data?*

(Old)The dataset needs a lot of pre processing. Current objective is to clean and remove redundancy and irrelevant columns. In this step we are actively discussing what our goal will be machine learning model, while understanding the data on a deeper level

In [118]:
# Excluding irrelevant columns
df = df[['Melding ID','Art - gruppe','Meldingsdato','Starttidspunkt','Havdybde start','Havdybde stopp','Rundvekt', 'Hovedområde start', 'Hovedart FAO', 'Trekkavstand']]


In [119]:
# Calculating the percentage of NaNs for each column to see whether we can afford to drop them
nanPercentage = df.isna().mean() * 100
print(nanPercentage)
df = df.dropna()
print("Current amount of rows: " + str(df.shape[0]))


Melding ID           0.000000
Art - gruppe         1.631122
Meldingsdato         0.000000
Starttidspunkt       0.000000
Havdybde start       0.000000
Havdybde stopp       0.000000
Rundvekt             1.629812
Hovedområde start    1.350210
Hovedart FAO         1.629812
Trekkavstand         0.007858
dtype: float64
Current amount of rows: 296645


In [120]:
# Collapsing certain columns?
#
#

In [121]:
# Many of the sea depth notations are positiv, which doesn't make sense.
# But the amount of them in relation to number of entries means it can't be discounted as an error
# In the lecture on fisheries it was mentioned that a lot fo these are inputed manually
# And that most of these non-sensical sea depths are actually correct, just lacking a minus.
# Therefore we are simply flipping all the positive sea depth into negatives.
df['Havdybde start'] = -df['Havdybde start'].abs()
df['Havdybde stopp'] = -df['Havdybde stopp'].abs()

In [122]:
df = df[df['Art - gruppe'].isin(['Torsk', 'Sei', 'Hyse'])]
df = df.pivot_table(index=['Melding ID', 'Meldingsdato', 'Starttidspunkt', 'Havdybde start', 'Havdybde stopp','Hovedområde start', 'Hovedart FAO', 'Trekkavstand'], columns='Art - gruppe', values='Rundvekt', aggfunc='sum').reset_index()
#?
df = df.fillna(0) # filling Nan values with 0
df['Hovedfangst'] = df[['Hyse', 'Sei', 'Torsk']].idxmax(axis=1)
print(f"Current dataset has {df.shape[1]} columns and {df.shape[0]} rows")


Current dataset has 12 columns and 65105 rows


## Testing decision trees and random forest
(Przybys) Having done this and some other test in another file I think we need to go back and look at the data again, because looking closer at "trekkavstand" its very bunched up, with a some extreme outliers. Maybe look at adding more feature into use, making a feature importance analysis and normalising them.

In [124]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score


X = df[['Trekkavstand','Havdybde stopp','Havdybde start']]
y = df['Hovedfangst']

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=32)
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.4863918412483873


In [125]:
from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=32)

random_forest.fit(X_train, y_train)
y_pred = random_forest.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.5623886465564908


In [129]:
from sklearn.model_selection import cross_val_score

tree = RandomForestClassifier(n_estimators=100, max_depth=30, random_state=32)
scores = cross_val_score(tree, X, y, cv=5)
print(scores, scores.mean(), scores.std())

[0.52499808 0.46240688 0.48321941 0.48391061 0.46524844] 0.4839566853544275 0.022357701541105612


#### Testing KNN

##### Predicting what the main intended catch is based on the largest rundvekt values.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Split the data into training and testing sets
X = df[['Hyse', 'Sei', 'Torsk']]
y = df['Hovedart FAO']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train the KNN classifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_scaled, y_train)

# Evaluate the model
accuracy = knn.score(X_test_scaled, y_test)
print("Accuracy:", accuracy)


#### Trying to predict dominant art based on dybde features 

In [ ]:
from sklearn.multioutput import MultiOutputClassifier
#using knn to predict the fish species based on depth
# Step 1: Prepare the data
X = df[['Havdybde start', 'Havdybde stopp']]
y = df[['Dominant Art']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_scaled, y_train)

y_pred = knn.predict(X_test_scaled)

accuracy = knn.score(X_test_scaled, y_test)
print("Accuracy:", accuracy)

#### Adding one hot encoded values 

In [ ]:
#converting Dominant Art column to one hot encoded values
dfEncoded = pd.get_dummies(df, columns=['Dominant Art'])
dfEncoded.head()

In [ ]:
#trying to predict the dominant art based on the depth again with one hot encoded values with multioutput, to see if any better
X = dfEncoded[['Havdybde start', 'Havdybde stopp']]
y = dfEncoded[['Dominant Art_Hyse', 'Dominant Art_Sei', 'Dominant Art_Torsk']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

knn = KNeighborsClassifier(n_neighbors=5)

classifier = MultiOutputClassifier(knn, n_jobs=-1)
classifier.fit(X_train_scaled, y_train)

y_pred = classifier.predict(X_test_scaled)

accuracy = classifier.score(X_test_scaled, y_test)
print("Accuracy:", accuracy)

#### Predicting only one dominant art with features dybde start + stopp

In [ ]:
X = dfEncoded[['Havdybde start', 'Havdybde stopp']]
y = dfEncoded[['Dominant Art_Hyse']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_scaled, y_train)

y_pred = knn.predict(X_test_scaled)

accuracy = knn.score(X_test_scaled, y_test)
print("Accuracy:", accuracy)